# LLM Chain




In [1]:
import streamlit as st
import docx2txt
import fitz
import urllib.request
from urllib.error import URLError, HTTPError
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

def read_docx(file_path):
    text = docx2txt.process(file_path)
    return text

def read_pdf(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"PDF file not found at: {file_path}")

    pdf_document = fitz.open(file_path)
    text = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        text += page.get_text()
    return text

def read_file(file):
    content = ""

    if file.type == "application/pdf":
        try:
            content = read_pdf(file.name)  # Assuming file.name contains the temporary path
        except FileNotFoundError as e:
            st.error(f"Error reading PDF file: {e}")
    elif file.type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
        content = read_docx(file)
    
    return content

def main():
    st.title("File Reader App")

    # Document title
    document_title = st.text_input("Enter the document title:")

    # File upload
    uploaded_file = st.file_uploader("Upload a document (doc, pdf) or provide a link", type=["docx", "pdf"])

    if uploaded_file is not None:
        try:
            # If a file is uploaded
            content = read_file(uploaded_file)
            # embeddings_model = OpenAIEmbeddings()
            # embeddings_model.embed_documents(content)
            
            text_splitter = CharacterTextSplitter(
                separator="\n\n\t\t",
                chunk_size=1000,
                chunk_overlap=200,
                length_function=len,
                is_separator_regex=False,
                keep_separator=True
            )
            metadatas = [{"title": document_title, "pageNo": 0}]

            texts = text_splitter.create_documents([content], metadatas=metadatas)
                
            st.write("### Content:")
            st.write(texts[11])
            # st.write(embeddings_model[:5])
        except FileNotFoundError as e:
            st.error(f"Error reading content from the uploaded file: {e}")

    # # Link input
    # link_input = st.text_input("Or provide a link to a document (doc, pdf)")
    # if st.button("Read from link"):
    #     if link_input:
    #         try:
    #             with urllib.request.urlopen(link_input) as response:
    #                 content = read_file(response)
    #                 st.write("### Content:")
    #                 st.write(content)
    #         except (URLError, HTTPError) as e:
    #             st.error(f"Error reading content from the provided link: {e}")

if __name__ == "__main__":
    main()


2024-01-19 12:45:30.191 
  command:

    streamlit run C:\Users\pixel6\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [40]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)

prompt = PromptTemplate(
    template = """ You are an experienced virologist Write a few sentences about the following {virus} in {language}. """,
    input_variables = ['virus', 'language']
)

chain = LLMChain(llm=llm, prompt=prompt)
output = chain.run({
    'virus': 'HSV', 'language': 'hindi'
})

print(output)

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "virus": "HSV",
  "language": "hindi"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human:  You are an experienced virologist Write a few sentences about the following HSV in hindi."
  ]
}
[llm/end] [1:chain:LLMChain > 2:llm:ChatOpenAI] [14.86s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "एचएसवी (हर्पीस सिम्प्लेक्स वायरस) एक वायरल संक्रमण है जो मनुष्यों में होता है। यह एक संक्रामक बीमारी है जिसका कारण हर्पीस सिम्प्लेक्स वायरस होता है। यह वायरस मुख्य तौर पर मुंह और जननांगों के आसपास की त्वचा को प्रभावित करता है और जल्दी ही बढ़ जाता है। यह वायरस आमतौर पर जलन, खुजली और छालों का कारण बनता है और इसका इलाज विशेषज्ञ चिकित्सक द्वारा किया जाना चाहिए।",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          

# Sequential Chain

In [73]:
from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAI

from langchain.chains import LLMChain, SimpleSequentialChain, TransformChain
from langchain import PromptTemplate

llm1 = OpenAI(temperature=0)
prompt1 = PromptTemplate(
    input_variables=['concept'],
    template='''You are an experienced scientist and Python programmer.
    Write a function that implements the concept of {concept}.'''
)
chain1 = LLMChain(llm=llm1, prompt=prompt1)

llm2 = ChatOpenAI(temperature=0)
prompt2 = PromptTemplate(
    input_variables=['function'],
    template='Given the Python function {function}, describe it as detailed as possible.'
)
chain2 = LLMChain(llm=llm2, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
output = overall_chain.run('softmax')

[chain/start] [1:chain:SimpleSequentialChain] Entering Chain run with input:
{
  "input": "softmax"
}
[chain/start] [1:chain:SimpleSequentialChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "concept": "softmax"
}
[llm/start] [1:chain:SimpleSequentialChain > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are an experienced scientist and Python programmer.\n    Write a function that implements the concept of softmax."
  ]
}
[llm/end] [1:chain:SimpleSequentialChain > 2:chain:LLMChain > 3:llm:OpenAI] [2.27s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\n\ndef softmax(x):\n    \"\"\"\n    Computes the softmax function for a given array of numbers.\n\n    Parameters:\n    x (array): Array of numbers.\n\n    Returns:\n    softmax (array): Array of softmax values for the input array.\n    \"\"\"\n    # Calculate the exponential of each element in the input array\n    exp_x = np.exp(x)\n\n    # Calculate t

# LongChain Chain